## Artificial Neural Networks 
### Case on : Churn Modeling ---> predict which customers churn or not

#### -- Installing Theano
#### -- pip install --upgrade --no-deps git+git://github.com/Theano/Theano.git

#### ------------------------------------------------------------ ####
#### -- Step 1: Installing Tensorflow
#### -- pip install tensorflow
#### -- Install Tensorflow from the website: https://www.tensorflow.org/install/pip#windows-native_1


#### ------------------------------------------------------------ ####


In [21]:
import warnings
warnings.filterwarnings('default')

In [2]:
!pip install tensorflow

#### ------------------------------------------------------------ ####
#### -- Step 2: Installing Keras
#### -- pip install --upgrade keras


In [1]:
!pip install Keras

In [22]:
#chack version
!pip --version

pip 22.2.2 from C:\Users\dipan\anaconda3\lib\site-packages\pip (python 3.9)



C:\Users\dipan\anaconda3\lib\threading.py:921: ResourceWarning: unclosed file <_io.BufferedWriter name=4>
  del self._target, self._args, self._kwargs
C:\Users\dipan\anaconda3\lib\threading.py:921: ResourceWarning: unclosed file <_io.BufferedReader name=5>
  del self._target, self._args, self._kwargs
C:\Users\dipan\anaconda3\lib\threading.py:921: ResourceWarning: unclosed file <_io.BufferedReader name=6>
  del self._target, self._args, self._kwargs
sys:1: ResourceWarning: unclosed socket <zmq.Socket(zmq.PUSH) at 0x1b920121d60>


In [1]:
# =============================================================================
# Importing the libraries
# =============================================================================
 
import numpy as np # Data Handling
import matplotlib.pyplot as plt # Data Visualization
import pandas as pd # # Data Handling
import os # Working Directory
from sklearn.preprocessing import LabelEncoder, OneHotEncoder # Transformation of Categorical columns into Numerical Values
from sklearn.compose import ColumnTransformer # Transformation same as level encoding and one hotencoding
from sklearn.model_selection import train_test_split # Splitting Data into Train & Test
from sklearn.preprocessing import StandardScaler # Neural Networks --> generally standarize the data
from sklearn.metrics import confusion_matrix # Model Evaluation 
from sklearn.metrics import classification_report # Model Evaluation 
import keras # Deep Learning Framework 
from keras.models import Sequential # Adding layers in the Neural Network 
from keras.layers import Dense # Adding layers in the Neural Network 

### 1. Reading the data

In [1]:
# =============================================================================
# Importing the dataset
# =============================================================================
#os.chdir(R'C:\Users\arpend.ganguly\OneDrive - Accenture\2021_DELL_ALL_Backup\02_G_IVY\Python\06NEURAL_NETWORKS')
#os.getcwd()
import pandas as pd
dataset = pd.read_csv('./data/Churn_Modelling.csv')
#print("The shape of data is", dataset.shape)
dataset.head()

#df = pd.read_csv('../data/listings.csv')

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [3]:
dataset.tail()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
9995,9996,15606229,Obijiaku,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,9997,15569892,Johnstone,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,9998,15584532,Liu,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,9999,15682355,Sabbatini,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1
9999,10000,15628319,Walker,792,France,Female,28,4,130142.79,1,1,0,38190.78,0


In [4]:
dataset.columns

Index(['RowNumber', 'CustomerId', 'Surname', 'CreditScore', 'Geography',
       'Gender', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard',
       'IsActiveMember', 'EstimatedSalary', 'Exited'],
      dtype='object')

In [5]:
dataset.Exited.value_counts() #distribution of churn or not
#1- :churn
#0 : not churn

0    7963
1    2037
Name: Exited, dtype: int64

In [6]:
dataset.Gender.value_counts() #distribution of Gender

Male      5457
Female    4543
Name: Gender, dtype: int64

In [7]:
 dataset.HasCrCard.value_counts() #distribution of cards

1    7055
0    2945
Name: HasCrCard, dtype: int64

In [8]:
dataset.IsActiveMember.value_counts() #distribution of active members

1    5151
0    4849
Name: IsActiveMember, dtype: int64

In [9]:
dataset.Geography.value_counts()

France     5014
Germany    2509
Spain      2477
Name: Geography, dtype: int64

In [10]:
dataset.Gender.value_counts()

Male      5457
Female    4543
Name: Gender, dtype: int64

In [11]:
dataset.describe()

,RowNumber,CustomerId,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
count,10000.00000,1.000000e+04,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.00000,10000.000000,10000.000000,10000.000000
mean,5000.50000,1.569094e+07,650.528800,38.921800,5.012800,76485.889288,1.530200,0.70550,0.515100,100090.239881,0.203700
std,2886.89568,7.193619e+04,96.653299,10.487806,2.892174,62397.405202,0.581654,0.45584,0.499797,57510.492818,0.402769
min,1.00000,1.556570e+07,350.000000,18.000000,0.000000,0.000000,1.000000,0.00000,0.000000,11.580000,0.000000
25%,2500.75000,1.562853e+07,584.000000,32.000000,3.000000,0.000000,1.000000,0.00000,0.000000,51002.110000,0.000000
50%,5000.50000,1.569074e+07,652.000000,37.000000,5.000000,97198.540000,1.000000,1.00000,1.000000,100193.915000,0.000000
75%,7500.25000,1.575323e+07,718.000000,44.000000,7.000000,127644.240000,2.000000,1.00000,1.000000,149388.247500,0.000000
max,10000.00000,1.581569e+07,850.000000,92.000000,10.000000,250898.090000,4.000000,1.00000,1.000000,199992.480000,1.000000


In [12]:
dataset.isnull().sum()

RowNumber          0
CustomerId         0
Surname            0
CreditScore        0
Geography          0
Gender             0
Age                0
Tenure             0
Balance            0
NumOfProducts      0
HasCrCard          0
IsActiveMember     0
EstimatedSalary    0
Exited             0
dtype: int64

### 2. Data Processing
--- i) Creating the **Final Feature Set and Dependent**

--- ii) Converting the **Categorical columns into Numerical ones**

--- iii) Applying the **Train Test Split**

--- iv) Applying the **Standarization across the dataframe**



In [13]:
# =============================================================================
# Data Pre-Processing
# =============================================================================

#1. Creating the Dependent and Independent Features
X = dataset.iloc[:, 3:13].values  # frist 3 columnns not requred thus we eleminated ie not counts.
y = dataset.iloc[:, 13].values  # only 13 columns dependent variable assigned to y
print("Feature Set:")
print(X.shape)
print(X)
print("Dependent Set:")
print(y)

Feature Set:
(10000, 10)
[[619 'France' 'Female' ... 1 1 101348.88]
 [608 'Spain' 'Female' ... 0 1 112542.58]
 [502 'France' 'Female' ... 1 0 113931.57]
 ...
 [709 'France' 'Female' ... 0 1 42085.58]
 [772 'Germany' 'Male' ... 1 0 92888.52]
 [792 'France' 'Female' ... 1 0 38190.78]]
Dependent Set:
[1 0 1 ... 1 1 0]


In [14]:
# Encoding categorical data through label encoding 
labelencoder_X_1 = LabelEncoder() # Geography Column 
X[:, 1] = labelencoder_X_1.fit_transform(X[:, 1])
labelencoder_X_2 = LabelEncoder() # Gender Column 
X[:, 2] = labelencoder_X_2.fit_transform(X[:, 2])
print("Feature Set post label encoding--->")
print(X.shape)
print(X)

Feature Set post label encoding--->
(10000, 10)
[[619 0 0 ... 1 1 101348.88]
 [608 2 0 ... 0 1 112542.58]
 [502 0 0 ... 1 0 113931.57]
 ...
 [709 0 0 ... 0 1 42085.58]
 [772 1 1 ... 1 0 92888.52]
 [792 0 0 ... 1 0 38190.78]]


In [15]:
type(X)

numpy.ndarray

In [16]:
#onehotencoder = OneHotEncoder(categorical_features = [1])
ct = ColumnTransformer([("Geography", OneHotEncoder(), [1])], remainder = 'passthrough')
X = ct.fit_transform(X)
X = X[:, 1:]
print(X.shape)
print(X)

(10000, 11)
[[0.0 0.0 619 ... 1 1 101348.88]
 [0.0 1.0 608 ... 0 1 112542.58]
 [0.0 0.0 502 ... 1 0 113931.57]
 ...
 [0.0 0.0 709 ... 0 1 42085.58]
 [1.0 0.0 772 ... 1 0 92888.52]
 [0.0 0.0 792 ... 1 0 38190.78]]


In [17]:
X_DT=pd.DataFrame(X)
X_DT.head()

,0,1,2,3,4,5,6,7,8,9,10
0,0.0,0.0,619,0,42,2,0.0,1,1,1,101348.88
1,0.0,1.0,608,0,41,1,83807.86,1,0,1,112542.58
2,0.0,0.0,502,0,42,8,159660.8,3,1,0,113931.57
3,0.0,0.0,699,0,39,1,0.0,2,0,0,93826.63
4,0.0,1.0,850,0,43,2,125510.82,1,1,1,79084.1


In [18]:
X_DT=pd.DataFrame(X)
X_DT.head()

,0,1,2,3,4,5,6,7,8,9,10
0,0.0,0.0,619,0,42,2,0.0,1,1,1,101348.88
1,0.0,1.0,608,0,41,1,83807.86,1,0,1,112542.58
2,0.0,0.0,502,0,42,8,159660.8,3,1,0,113931.57
3,0.0,0.0,699,0,39,1,0.0,2,0,0,93826.63
4,0.0,1.0,850,0,43,2,125510.82,1,1,1,79084.1


In [35]:
# Splitting the dataset into the Training set and Test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

# Feature Scaling for faster computation
sc = StandardScaler() # transfer the standerised data if do not sc data the train requred lot of time.
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)
print("Standarized X Train --->")
print(X_train)
print("Standarized X Test --->")
print(X_test)
print("Data Transformations for Neural Networks is complete! ")

Standarized X Train --->
[[-0.5698444   1.74309049  0.16958176 ...  0.64259497 -1.03227043
   1.10643166]
 [ 1.75486502 -0.57369368 -2.30455945 ...  0.64259497  0.9687384
  -0.74866447]
 [-0.5698444  -0.57369368 -1.19119591 ...  0.64259497 -1.03227043
   1.48533467]
 ...
 [-0.5698444  -0.57369368  0.9015152  ...  0.64259497 -1.03227043
   1.41231994]
 [-0.5698444   1.74309049 -0.62420521 ...  0.64259497  0.9687384
   0.84432121]
 [ 1.75486502 -0.57369368 -0.28401079 ...  0.64259497 -1.03227043
   0.32472465]]
Standarized X Test --->
[[ 1.75486502 -0.57369368 -0.55204276 ...  0.64259497  0.9687384
   1.61085707]
 [-0.5698444  -0.57369368 -1.31490297 ...  0.64259497 -1.03227043
   0.49587037]
 [-0.5698444   1.74309049  0.57162971 ...  0.64259497  0.9687384
  -0.42478674]
 ...
 [-0.5698444   1.74309049 -0.74791227 ...  0.64259497 -1.03227043
   0.71888467]
 [ 1.75486502 -0.57369368 -0.00566991 ...  0.64259497  0.9687384
  -1.54507805]
 [ 1.75486502 -0.57369368 -0.79945688 ...  0.64259497 

### 3. Creating the Neural Network Architecture

In [36]:
# =============================================================================
# Building the Artifical Neural Network
# =============================================================================

# =============================================================================
# We can summarize 7 steps:
# 1. Randomly initialise the weights to small numbers close to 0, but not 0
# 2. Input the first observation of your dataset, in the input layer, each feature in one input node
# 3. Forward-Propagation: from left to right, the neurons are activated in a way the impact of each neuron’s activation is limited by the weights and it runs until getting the y
# 4. Compare the predicted result to the actual result. Measure the error generated
# 5. Back-propagation: from right to left. The error is back propagated. Update the weights according to how much they are responsible for the error. The learning rate decides how much we update the weights
# 6. Repeat steps 1 to 5 and update the weights after each observation (Reinforcement Learning). Repeat steps 1 to 5 but update the weights only after a batch of observation (Batch Learning)
# 7. When the whole training set passed through the ANN, that makes an epoch. 
# =============================================================================

#nodes of hidden layers, based on experiments, is to choose the average between the input and the output layers. 

In [54]:
# Initialising the ANN
classifier = Sequential() # this is empty newral networks

# Adding the input layer and the first hidden layer
classifier.add(Dense(6, kernel_initializer= 'uniform', activation = 'relu', input_dim = 11))

# Adding the second hidden layer
classifier.add(Dense(6, kernel_initializer=  'uniform', activation = 'relu'))

# Adding the output layer
classifier.add(Dense(1, kernel_initializer='uniform', activation = 'sigmoid'))

#---------------------------------------------------------------------->
#Output_dim = nodes of first hidden layer
#init = initialization of the weights based on an uniform distribution
#activation = activation function
#input_dim = no. of input features
#---------------------------------------------------------------------->

#### 4. Compiling the Architecture

In [55]:
# Compiling the ANN
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
#Binary Cross entropy since we are focussing on Output class [0,1]
# Fitting the ANN to the Training set
classifier.fit(X_train, y_train, batch_size = 10, epochs = 20)

Epoch 1/20
800/800 [==============================] - 2s 1ms/step - loss: 0.4781 - accuracy: 0.7955
Epoch 2/20
800/800 [==============================] - 1s 1ms/step - loss: 0.4282 - accuracy: 0.7960
Epoch 3/20
800/800 [==============================] - 1s 1ms/step - loss: 0.4216 - accuracy: 0.8061
Epoch 4/20
800/800 [==============================] - 1s 1ms/step - loss: 0.4179 - accuracy: 0.8253
Epoch 5/20
800/800 [==============================] - 1s 1ms/step - loss: 0.4153 - accuracy: 0.8298
Epoch 6/20
800/800 [==============================] - 1s 1ms/step - loss: 0.4139 - accuracy: 0.8313
Epoch 7/20
800/800 [==============================] - 1s 1ms/step - loss: 0.4122 - accuracy: 0.8319
Epoch 8/20
800/800 [==============================] - 1s 1ms/step - loss: 0.4119 - accuracy: 0.8320
Epoch 9/20
800/800 [==============================] - 1s 1ms/step - loss: 0.4104 - accuracy: 0.8331
Epoch 10/20
800/800 [==============================] - 1s 1ms/step - loss: 0.4094 - accuracy: 0.8355

#### 5. Making the Final Predictions

In [56]:
# Part 3 - Making the predictions and evaluating the model

# Predicting the Test set results
y_pred = classifier.predict(X_test)
y_pred = (y_pred > 0.5)#Converting 1, 0 Class

# # Making the Confusion Matrix
cm = confusion_matrix(y_test, y_pred)
print(classification_report(y_pred, 
            y_test))

63/63 [==============================] - 0s 1ms/step
              precision    recall  f1-score   support

       False       0.97      0.85      0.91      1828
        True       0.32      0.76      0.45       172

    accuracy                           0.84      2000
   macro avg       0.65      0.81      0.68      2000
weighted avg       0.92      0.84      0.87      2000



In [29]:
y_test

array([0, 1, 0, ..., 0, 1, 0], dtype=int64)

In [30]:
y_test_df=pd.DataFrame({'y_test_value':y_test})

In [31]:
y_test_df.y_test_value.value_counts()

0    3966
1    1034
Name: y_test_value, dtype: int64

In [32]:
1595/(1595+405)

0.7975